In [11]:
# !pip install konlpy

In [13]:
# !pip install transformers

In [1]:
import boto3
import requests
import json
import datetime
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from io import StringIO
from hanspell import spell_checker
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

In [2]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer
from keras.utils import pad_sequences

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-06-24 08:05:33.820051: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-24 08:05:33.820084: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
PATH = './'
DATE = int(str(datetime.datetime.now() - datetime.timedelta(1))[:10].replace('-', ''))

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cpu()
model.load_state_dict(torch.load(PATH + 'model_crawling_bert.pt', map_location=torch.device('cpu')))
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [5]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&' 
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2",
                 "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 
                 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi'} 

In [11]:
okt = Okt()

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

stop_words = open('korean_stopwords.txt').read()
stop_words = stop_words.split('\n')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
def upload_file_s3(bucket, file, content):
    try:
        csv_buffer = StringIO()
        content.to_csv(csv_buffer)
        s3_resource = boto3.resource('s3')
        s3_resource.Object(bucket, file).put(Body=csv_buffer.getvalue())
        return True
    except:
        return False

In [13]:
def ls_s3(bucket, folder):
    resource = boto3.resource('s3')
    try:
        for obj in resource.Bucket(bucket).objects.filter(Prefix=f'{folder}/'):
            print(obj.key)
        return True
    except:
        return False

In [14]:
def download_file_s3(bucket, folder, file, path='./'):
    client = boto3.client('s3')
    file = client.get_object(Bucket=bucket, Key=f'{folder}/{file}')
    body = file['Body']
    csv_string = body.read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_string))
    return df

In [15]:
def get_top10_movies():
    url = f'http://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=5d450765c401dac01475905006e44d99&targetDt={DATE}'
    res = requests.get(url)
    txt = res.text
    result = json.loads(txt)

    movie_titles = []
    for movie_info in result['boxOfficeResult']['dailyBoxOfficeList']:
        movie_titles.append(movie_info['movieNm'])
    
    review_urls = []
    movie_codes = []
    for movie_title in movie_titles:
        review_url, movie_code = get_review_url(movie_title)
        review_urls.append(review_url)
        movie_codes.append(movie_code)
    
    return movie_titles, review_urls, movie_codes

In [16]:
def get_review_url(movie_title):
    url = f'https://movie.naver.com/movie/search/result.naver?query={movie_title}&section=all&ie=utf8'
    res = requests.get(url)
    
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'html.parser')
        user_dic = {}
        idx = 1
        for href in soup.find("ul", class_="search_list_1").find_all("li"):
            user_dic[idx] = int(href.dl.dt.a['href'][30:])
            idx += 1
        
        movie_code = user_dic[1]
        order_key = 'newest'  # 공감순: order=sympathyScore, 최신순: order=newest
        url = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={movie_code}&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order={order_key}&page='
        return url, movie_code
    else:
        print(f'Request Error: Code {res.status_code}')
        return '', 0

In [17]:
def get_img_src(movie_code):
    url = f'https://movie.naver.com/movie/bi/mi/basic.naver?code={movie_code}'
    res = requests.get(url)
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'html.parser')
        img_src = soup.select_one('#content > div.article > div.mv_info_area > div.poster > a > img').attrs['src']
        return img_src
    else:
        print(f'Request Error: Code {res.status_code}')
        return ''

In [18]:
def convert_input_data(sentences):
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    MAX_LEN = 128
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    attention_masks = []
    for seq in input_ids:
        seq_mask = [float(i > 0) for i in seq]
        attention_masks.append(seq_mask)

    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)
    return inputs, masks

In [19]:
def test_sentences(sentences):
    model.eval()
    inputs, masks = convert_input_data(sentences)
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    return logits

In [39]:
def analysis(df):
    # 각 리뷰에 대해 긍부정 라벨링
    df['Label'] = np.nan
    for i in tqdm(range(len(df))):
        try:
            logits = test_sentences([df.loc[i, '리뷰']])
            df.loc[i, 'Label'] = np.argmax(logits)
        except:
            pass
    
    return df

In [21]:
def clean_punc(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
        specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''} 

    for s in specials:
        text = text.replace(s, specials[s])
   
    return text.strip()

In [22]:
def clean_text(texts):
    corpus = []
    for i in range(len(texts)):
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation
        review = re.sub(r'\d+','', str(texts[i]))# remove number
        review = review.lower() #lower case
        review = re.sub(r'\s+', ' ', review) #remove extra space
        review = re.sub(r'<[^>]+>','',review) #remove Html tags
        review = re.sub(r'\s+', ' ', review) #remove spaces
        review = re.sub(r"^\s+", '', review) #remove space from start
        review = re.sub(r'\s+$', '', review) #remove space from the end
        review = re.sub(r'♥','',review)
        review = re.sub(r'~','',review)
        review = re.sub(r'&','',review)
        corpus.append(review)
    
    return corpus

In [23]:
def grammar_check(text):
    spelled_sent = spell_checker.check(text)
    hanspell_sent = spelled_sent.checked
    return hanspell_sent

In [24]:
def tokenize_tagged(text):
    temp_X = okt.morphs(text, norm=True, stem=False) # 토큰화
    temp_X = [word for word in temp_X if word not in stop_words] # 불용어 제거
    return " ".join(temp_X)

In [25]:
def preprocess(df):
    df['전처리완료'] = ''
    for i in tqdm(range(len(df))):
        if type(df.loc[i, '리뷰']) == str:
            text = df.loc[i, '리뷰']
            text1 = clean_punc(text, punct, punct_mapping)
            text2 = ''.join(clean_text(text1))
            text3 = grammar_check(text2)
            text4 = tokenize_tagged(text3)
            df.loc[i, '전처리완료'] = text4
    
    return df

---
## 1. S3 (movietomodel)에서 해당 DATE의 REVIEW csv 파일 로드 (word/review_{DATE}.csv)

## 2. 감성분석 & 이미지 소스 & 전처리

## 3. S3 (modeltoelastic)에 결과 업로드 (word/test_{DATE}.csv)

In [30]:
DATE = '20220622'

In [26]:
print(DATE)

20220623


In [31]:
MOVIE_TITLES, REVIEW_URLS, MOVIE_CODES = get_top10_movies()
print(MOVIE_TITLES)
print(REVIEW_URLS)
print(MOVIE_CODES)

['탑건: 매버릭', '마녀(魔女) Part2. The Other One', '범죄도시 2', '브로커', '버즈 라이트이어', '쥬라기 월드: 도미니언', '룸 쉐어링', '니 부모 얼굴이 보고 싶다', '뜨거운 피', '로스트 시티']
['https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=81888&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=newest&page=', 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=196367&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=newest&page=', 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=192608&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=newest&page=', 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=196854&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=newest&page=', 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=214603&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=newest&page=', 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=191646&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=newest&page

In [32]:
IMAGE_SRCS = [get_img_src(movie_code) for movie_code in MOVIE_CODES]
print(IMAGE_SRCS)

['https://movie-phinf.pstatic.net/20220509_176/1652081912471yhg3N_JPEG/movie_image.jpg?type=m203_290_2', 'https://movie-phinf.pstatic.net/20220615_63/1655270906406BGdFF_JPEG/movie_image.jpg?type=m203_290_2', 'https://movie-phinf.pstatic.net/20220516_144/1652665409592Chvey_JPEG/movie_image.jpg?type=m203_290_2', 'https://movie-phinf.pstatic.net/20220511_189/1652251073330PXNoG_JPEG/movie_image.jpg?type=m203_290_2', 'https://movie-phinf.pstatic.net/20220616_274/1655363103107UB9nG_JPEG/movie_image.jpg?type=m203_290_2', 'https://movie-phinf.pstatic.net/20220516_144/1652687286550mcE4G_JPEG/movie_image.jpg?type=m203_290_2', 'https://movie-phinf.pstatic.net/20220608_157/1654654613441kSez5_JPEG/movie_image.jpg?type=m203_290_2', 'https://movie-phinf.pstatic.net/20220429_150/1651196758429nQ4v4_JPEG/movie_image.jpg?type=m203_290_2', 'https://movie-phinf.pstatic.net/20220324_247/1648097201748E1Kzv_JPEG/movie_image.jpg?type=m203_290_2', 'https://movie-phinf.pstatic.net/20220422_222/16506049712319Ya0u

In [33]:
review_df = download_file_s3('movietomodel', 'word', f'review_{DATE}.csv').drop('Unnamed: 0', axis=1)
review_df

,제목,리뷰,별점
0,탑건: 매버릭,그저 가슴이 웅장해진다..,10
1,탑건: 매버릭,재밌게 봤어요 ㅠㅠ!!!,10
2,탑건: 매버릭,개봉날 아침에 조조 보고 밤에 또 보고 왔네요. 개인적으로 screen x나 아이맥...,10
3,탑건: 매버릭,개봉날 아침에 조조 보고 밤에 또 보고 왔네요. 개인적으로 screen x나 아이맥...,10
4,탑건: 매버릭,와...인생영화입니다.박수도 눈치껏 치고...최고입니다!!!,10
...,...,...,...
2635,니 부모 얼굴이 보고 싶다,"몰입도 높은 영화였음.연출, 연기 모두 좋았음.추천!",8
2636,뜨거운 피,배우들 연기 좋아야 되는건 당연 한거고 스토리가 흡...,6
2637,뜨거운 피,정우 더럽게 안 풀린다 다음 작품은 대박나길,6
2638,뜨거운 피,하여간 경상도 사투리 영화는 자막 없으면 뭐라고 씨부리는 건지 알 수가 없다.,1


In [38]:
review_df = analysis(review_df)
review_df

,제목,리뷰,별점,Label
0,탑건: 매버릭,그저 가슴이 웅장해진다..,10,1.0
1,탑건: 매버릭,재밌게 봤어요 ㅠㅠ!!!,10,1.0
2,탑건: 매버릭,개봉날 아침에 조조 보고 밤에 또 보고 왔네요. 개인적으로 screen x나 아이맥...,10,1.0
3,탑건: 매버릭,개봉날 아침에 조조 보고 밤에 또 보고 왔네요. 개인적으로 screen x나 아이맥...,10,1.0
4,탑건: 매버릭,와...인생영화입니다.박수도 눈치껏 치고...최고입니다!!!,10,1.0
...,...,...,...,...
2635,니 부모 얼굴이 보고 싶다,"몰입도 높은 영화였음.연출, 연기 모두 좋았음.추천!",8,1.0
2636,뜨거운 피,배우들 연기 좋아야 되는건 당연 한거고 스토리가 흡...,6,1.0
2637,뜨거운 피,정우 더럽게 안 풀린다 다음 작품은 대박나길,6,0.0
2638,뜨거운 피,하여간 경상도 사투리 영화는 자막 없으면 뭐라고 씨부리는 건지 알 수가 없다.,1,1.0


In [40]:
review_df = preprocess(review_df)
review_df

100%|███████████████████████████████████████| 2640/2640 [04:04<00:00, 10.80it/s]


,제목,리뷰,별점,Label,전처리완료
0,탑건: 매버릭,그저 가슴이 웅장해진다..,10,1.0,가슴 웅장 해진다
1,탑건: 매버릭,재밌게 봤어요 ㅠㅠ!!!,10,1.0,재밌게 봤어요 ㅠㅠ
2,탑건: 매버릭,개봉날 아침에 조조 보고 밤에 또 보고 왔네요. 개인적으로 screen x나 아이맥...,10,1.0,개봉 날 아침 조조 보고 밤 보고 왔네요 개인 적 screenx 아이맥스 매우 추천...
3,탑건: 매버릭,개봉날 아침에 조조 보고 밤에 또 보고 왔네요. 개인적으로 screen x나 아이맥...,10,1.0,개봉 날 아침 조조 보고 밤 보고 왔네요 개인 적 screenx 아이맥스 매우 추천...
4,탑건: 매버릭,와...인생영화입니다.박수도 눈치껏 치고...최고입니다!!!,10,1.0,인생 영화 박수 눈치 껏 치고 최고
...,...,...,...,...,...
2635,니 부모 얼굴이 보고 싶다,"몰입도 높은 영화였음.연출, 연기 모두 좋았음.추천!",8,1.0,몰입도 높은 영화 였음 연출 연기 좋았음 추천
2636,뜨거운 피,배우들 연기 좋아야 되는건 당연 한거고 스토리가 흡...,6,1.0,배우 연기 좋아야 되는 건 당연한 거고 스토리 흡
2637,뜨거운 피,정우 더럽게 안 풀린다 다음 작품은 대박나길,6,0.0,정우 더럽게 안 풀린다다 음 작품 대박 나길
2638,뜨거운 피,하여간 경상도 사투리 영화는 자막 없으면 뭐라고 씨부리는 건지 알 수가 없다.,1,1.0,하여간 경상도 사투리 영화 자막 없으면 뭐라고 씨불 건지 알 수가 없다


In [41]:
review_df['사진'] = review_df['제목'].apply(lambda x: IMAGE_SRCS[MOVIE_TITLES.index(x)])
review_df

,제목,리뷰,별점,Label,전처리완료,사진
0,탑건: 매버릭,그저 가슴이 웅장해진다..,10,1.0,가슴 웅장 해진다,https://movie-phinf.pstatic.net/20220509_176/1...
1,탑건: 매버릭,재밌게 봤어요 ㅠㅠ!!!,10,1.0,재밌게 봤어요 ㅠㅠ,https://movie-phinf.pstatic.net/20220509_176/1...
2,탑건: 매버릭,개봉날 아침에 조조 보고 밤에 또 보고 왔네요. 개인적으로 screen x나 아이맥...,10,1.0,개봉 날 아침 조조 보고 밤 보고 왔네요 개인 적 screenx 아이맥스 매우 추천...,https://movie-phinf.pstatic.net/20220509_176/1...
3,탑건: 매버릭,개봉날 아침에 조조 보고 밤에 또 보고 왔네요. 개인적으로 screen x나 아이맥...,10,1.0,개봉 날 아침 조조 보고 밤 보고 왔네요 개인 적 screenx 아이맥스 매우 추천...,https://movie-phinf.pstatic.net/20220509_176/1...
4,탑건: 매버릭,와...인생영화입니다.박수도 눈치껏 치고...최고입니다!!!,10,1.0,인생 영화 박수 눈치 껏 치고 최고,https://movie-phinf.pstatic.net/20220509_176/1...
...,...,...,...,...,...,...
2635,니 부모 얼굴이 보고 싶다,"몰입도 높은 영화였음.연출, 연기 모두 좋았음.추천!",8,1.0,몰입도 높은 영화 였음 연출 연기 좋았음 추천,https://movie-phinf.pstatic.net/20220429_150/1...
2636,뜨거운 피,배우들 연기 좋아야 되는건 당연 한거고 스토리가 흡...,6,1.0,배우 연기 좋아야 되는 건 당연한 거고 스토리 흡,https://movie-phinf.pstatic.net/20220324_247/1...
2637,뜨거운 피,정우 더럽게 안 풀린다 다음 작품은 대박나길,6,0.0,정우 더럽게 안 풀린다다 음 작품 대박 나길,https://movie-phinf.pstatic.net/20220324_247/1...
2638,뜨거운 피,하여간 경상도 사투리 영화는 자막 없으면 뭐라고 씨부리는 건지 알 수가 없다.,1,1.0,하여간 경상도 사투리 영화 자막 없으면 뭐라고 씨불 건지 알 수가 없다,https://movie-phinf.pstatic.net/20220324_247/1...


In [42]:
review_df.to_csv("tmp.csv", index=False)

In [43]:
upload_file_s3('modeltoelastic', f'word/test_{DATE}.csv', review_df)

True

In [44]:
download_file_s3('modeltoelastic', 'word', f'test_{DATE}.csv')

,Unnamed: 0,제목,리뷰,별점,Label,전처리완료,사진
0,0,탑건: 매버릭,그저 가슴이 웅장해진다..,10,1.0,가슴 웅장 해진다,https://movie-phinf.pstatic.net/20220509_176/1...
1,1,탑건: 매버릭,재밌게 봤어요 ㅠㅠ!!!,10,1.0,재밌게 봤어요 ㅠㅠ,https://movie-phinf.pstatic.net/20220509_176/1...
2,2,탑건: 매버릭,개봉날 아침에 조조 보고 밤에 또 보고 왔네요. 개인적으로 screen x나 아이맥...,10,1.0,개봉 날 아침 조조 보고 밤 보고 왔네요 개인 적 screenx 아이맥스 매우 추천...,https://movie-phinf.pstatic.net/20220509_176/1...
3,3,탑건: 매버릭,개봉날 아침에 조조 보고 밤에 또 보고 왔네요. 개인적으로 screen x나 아이맥...,10,1.0,개봉 날 아침 조조 보고 밤 보고 왔네요 개인 적 screenx 아이맥스 매우 추천...,https://movie-phinf.pstatic.net/20220509_176/1...
4,4,탑건: 매버릭,와...인생영화입니다.박수도 눈치껏 치고...최고입니다!!!,10,1.0,인생 영화 박수 눈치 껏 치고 최고,https://movie-phinf.pstatic.net/20220509_176/1...
...,...,...,...,...,...,...,...
2635,2635,니 부모 얼굴이 보고 싶다,"몰입도 높은 영화였음.연출, 연기 모두 좋았음.추천!",8,1.0,몰입도 높은 영화 였음 연출 연기 좋았음 추천,https://movie-phinf.pstatic.net/20220429_150/1...
2636,2636,뜨거운 피,배우들 연기 좋아야 되는건 당연 한거고 스토리가 흡...,6,1.0,배우 연기 좋아야 되는 건 당연한 거고 스토리 흡,https://movie-phinf.pstatic.net/20220324_247/1...
2637,2637,뜨거운 피,정우 더럽게 안 풀린다 다음 작품은 대박나길,6,0.0,정우 더럽게 안 풀린다다 음 작품 대박 나길,https://movie-phinf.pstatic.net/20220324_247/1...
2638,2638,뜨거운 피,하여간 경상도 사투리 영화는 자막 없으면 뭐라고 씨부리는 건지 알 수가 없다.,1,1.0,하여간 경상도 사투리 영화 자막 없으면 뭐라고 씨불 건지 알 수가 없다,https://movie-phinf.pstatic.net/20220324_247/1...


---

In [71]:
def run(path='./'):
    PATH = path
    DATE = int(str(datetime.datetime.now() - datetime.timedelta(1))[:10].replace('-', ''))
    MOVIE_TITLES, REVIEW_URLS, MOVIE_CODES = get_top10_movies()
    IMAGE_SRCS = [get_img_src(movie_code) for movie_code in MOVIE_CODES]
    
    review_df = download_file_s3('movietomodel', 'word', f'review_{DATE}.csv').drop('Unnamed: 0', axis=1)
    review_df = preprocess(review_df)
    upload_file_s3('modeltoelastic', f'word/test_{DATE}.csv', review_df)

In [72]:
run()

100%|███████████████████████████████████████| 2640/2640 [03:55<00:00, 11.23it/s]


In [73]:
download_file_s3('modeltoelastic', 'word', f'test_{DATE}.csv')

,Unnamed: 0,제목,리뷰,별점,전처리완료
0,0,탑건: 매버릭,그저 가슴이 웅장해진다..,10,가슴 웅장 해진다
1,1,탑건: 매버릭,재밌게 봤어요 ㅠㅠ!!!,10,재밌게 봤어요 ㅠㅠ
2,2,탑건: 매버릭,개봉날 아침에 조조 보고 밤에 또 보고 왔네요. 개인적으로 screen x나 아이맥...,10,개봉 날 아침 조조 보고 밤 보고 왔네요 개인 적 screenx 아이맥스 매우 추천...
3,3,탑건: 매버릭,개봉날 아침에 조조 보고 밤에 또 보고 왔네요. 개인적으로 screen x나 아이맥...,10,개봉 날 아침 조조 보고 밤 보고 왔네요 개인 적 screenx 아이맥스 매우 추천...
4,4,탑건: 매버릭,와...인생영화입니다.박수도 눈치껏 치고...최고입니다!!!,10,인생 영화 박수 눈치 껏 치고 최고
...,...,...,...,...,...
2635,2635,니 부모 얼굴이 보고 싶다,"몰입도 높은 영화였음.연출, 연기 모두 좋았음.추천!",8,몰입도 높은 영화 였음 연출 연기 좋았음 추천
2636,2636,뜨거운 피,배우들 연기 좋아야 되는건 당연 한거고 스토리가 흡...,6,배우 연기 좋아야 되는 건 당연한 거고 스토리 흡
2637,2637,뜨거운 피,정우 더럽게 안 풀린다 다음 작품은 대박나길,6,정우 더럽게 안 풀린다다 음 작품 대박 나길
2638,2638,뜨거운 피,하여간 경상도 사투리 영화는 자막 없으면 뭐라고 씨부리는 건지 알 수가 없다.,1,하여간 경상도 사투리 영화 자막 없으면 뭐라고 씨불 건지 알 수가 없다
